In [1]:
#!pip install -r requirements.txt

In [2]:
import yaml
import timeit
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from utility import *
from ultralytics import settings
from registry import get_model, get_pipeline
#settings.update({'datasets_dir': os.path.abspath('.')})


Install mish-cuda to speed up training and inference. More importantly, replace the naive Mish with MishCuda will give a ~1.5G memory saving during training.
c:\Users\user\anaconda3\envs\AerialObjectDetection\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\user\anaconda3\envs\AerialObjectDetection\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
C:\Users\user\Desktop\Aerial Object Detection\AERIAL_OBJECT_DETECTION\Models\YOLOH\models\backbone\convnext.py:204: UserWarning: Overwriting convnext_tiny in registry with models.backbone.convnext.convnext_tiny. This is because the name bei

In [ ]:
project_name = 'Object Detection'
# dataset_root 가 상대경로일 경우 colab 환경에서는 무조건 앞에 datasets 를 붙여서 찾는 문제가 있음
dataset_root = 'Datasets'
output_dir = 'output'
dataset_names = ['COCO_airplane']
model_names = ['YoloOW', 'yoloh18', 'yoloh50', 'yoloh101' ]
iterations = [1, 5]
epochs = 1
early_stop = 20
batch_size = 16
image_size = 640
optimizer = 'AdamW'
lr = 1e-3
momentum = 0.9
weight_decay = 1e-4
optim_args = {'optimizer': optimizer, 'lr': lr, 'momentum': momentum, 'weight_decay': weight_decay}
devices = [0]
device = torch.device("cuda:"+str(devices[0])) if len(devices)>0 else torch.device("cpu")
train_split = 0.6
val_split = 0.2
test_split = 0.2
num_workers = 2

In [ ]:
from utility.utils import control_random_seed, update_dataset_paths, format_measures, merge_and_update_df


Experiments_Time = datetime.now().strftime("%y%m%d_%H%M%S")
print('Experiment Start Time:',Experiments_Time)

ex_dict = {}
ex_dict['Experiment Time'] = Experiments_Time;ex_dict['Epochs'] = epochs;
ex_dict['Batch Size'] = batch_size;
ex_dict['Early Stop'] = early_stop;
ex_dict['Device'] = device
ex_dict['Optimizer'] = optimizer;
ex_dict['LR']=optim_args['lr']; ex_dict['Weight Decay']=optim_args['weight_decay'];ex_dict['Momentum']=optim_args['momentum'];
#ex_dict['Hyp'] = dict(lr0=ex_dict['LR'], momentum=ex_dict['Momentum'], weight_decay=ex_dict['Weight Decay'])
ex_dict['Image Size'] = image_size
ex_dict['Output Dir'] = output_dir
ex_dict['Num Workers'] = num_workers

for iteration in range(iterations[0], iterations[1]+1):
    print(f'(Iter {iteration})')
    seed = iteration
    ex_dict['Iteration'] = iteration
    for j, Dataset_Name in enumerate(dataset_names):
        print(f'Dataset: {Dataset_Name} ({j+1}/{len(dataset_names)})');
        control_random_seed(seed)
        data_yaml_path = f"{dataset_root}/{Dataset_Name}/data_iter_{iteration:02d}.yaml"
        with open(data_yaml_path, 'r') as f:
            data_config = yaml.load(f, Loader=yaml.FullLoader)
        ex_dict['Dataset Name'] = Dataset_Name; ex_dict['Data Config'] = data_yaml_path; ex_dict['Number of Classes'] = data_config['nc']; ex_dict['Class Names'] = data_config['names'];
        ex_dict['Split Size'] = f"{data_config['train_count']}/{data_config['val_count']}/{data_config['test_count']}"
        update_dataset_paths(dataset_root, Dataset_Name, iteration)
        
        for k, model_name in enumerate(model_names):
            print(f'{model_name} ({k+1}/{len(model_names)}) (Iter {iteration})', end=' ')
            print(f'Dataset: {Dataset_Name} ({j+1}/{len(dataset_names)})', end=' ')
            control_random_seed(seed)
            
            model = get_model(model_name, ex_dict)
            train_fn, eval_fn, test_fn = get_pipeline(model_name)

            ex_dict['Model Name'] = model_name; ex_dict['Model']=model;
            start = timeit.default_timer()

            ex_dict = train_fn(ex_dict)
            ex_dict = eval_fn(ex_dict)
            ex_dict = test_fn(ex_dict)

            ex_dict['Train-Test Time'] = timeit.default_timer() - start
            #################
            if ex_dict.get('Test Results') is None:
                raise RuntimeError("평가 결과가 없습니다. 위의 체크리스트를 참고해 원인부터 해결하세요.")

            eval_dict = format_measures(ex_dict.get('Test Results'))
            output_csv = f"{ex_dict['Experiment Time']}_{project_name}_Results.csv"
            merge_and_update_df(ex_dict, eval_dict, output_csv, exclude_columns=['Model', 'Train Results', 'Test Results'])

Experiment Start Time: 250427_221821
(Iter 1)
Dataset: COCO_airplane (1/1)
YoloOW (1/4) (Iter 1) Dataset: COCO_airplane (1/1) [YoloOW] Using config → C:\Users\user\Desktop\Aerial Object Detection\AERIAL_OBJECT_DETECTION\Models\YoloOW\cfg\training\yoloOW.yaml

[trainer] 🔍 Checking anchor fit to dataset...
[YOLOTxtDataset] Caching labels for anchor check...
[trainer] ⚠️ Low anchor fit (0.12 < 4.0). Consider running autoanchor.
Optimizer: adamw
--momentum: 0.9
--weight_decay: 0.0001


[YoloOW] Epoch 1/5:   0%|          | 0/41 [00:00<?, ?it/s]


[DEBUG] first batch target rows = 16
[DEBUG] first 5 targets:
 tensor([[0.00000, 0.00000, 0.36098, 0.40695, 0.51108, 0.28026],
        [1.00000, 0.00000, 0.44410, 0.28651, 0.88483, 0.28090],
        [2.00000, 0.00000, 0.49249, 0.51572, 0.96752, 0.71235],
        [3.00000, 0.00000, 0.49597, 0.50713, 0.90334, 0.38318],
        [4.00000, 0.00000, 0.58469, 0.24538, 0.83061, 0.49075]])


c:\Users\user\Desktop\Aerial Object Detection\AERIAL_OBJECT_DETECTION\utility\trainer.py:114: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\Scalar.cpp:23.)
  total_loss += loss.item()
[YoloOW] Epoch 1/5: 100%|██████████| 41/41 [00:39<00:00,  1.03it/s, Loss=2.1312, box=0.123, obj=0.010, cls=0.000, total loss=0.133, lr=0.000195]



[YoloOW] ✅ Epoch 1/5 complete in 46.64s | Avg Loss: 1.7072

[YoloOW] 📊 Running evaluation after epoch 1 ...
[validate_yoloow_model] ✅  Avg Loss  (box/obj/cls/total): 0.0940 / 0.0134 / 0.0000 / 0.1074


[YoloOW] Epoch 2/5:  39%|███▉      | 16/41 [00:22<00:34,  1.38s/it, Loss=1.3961, box=0.079, obj=0.009, cls=0.000, total loss=0.087, lr=0.000244]

In [ ]:
import torch

print(f"CUDA 사용 가능 여부: {torch.cuda.is_available()}")
print(f"CUDA 디바이스 수: {torch.cuda.device_count()}")
print(f"현재 사용 디바이스: {torch.cuda.current_device()}")
print(f"디바이스 이름: {torch.cuda.get_device_name(0)}")
print(f"PyTorch CUDA 버전: {torch.version.cuda}")
print(f"PyTorch 버전: {torch.__version__}")


CUDA 사용 가능 여부: True
CUDA 디바이스 수: 1
현재 사용 디바이스: 0
디바이스 이름: NVIDIA GeForce RTX 5060 Ti
PyTorch CUDA 버전: 12.8
PyTorch 버전: 2.8.0.dev20250420+cu128


cat: /root/.config/Ultralytics/settings.json: No such file or directory
